## 2D FCS

### Description

We run a monte carlo simulation of the spectroscopy measurement and compare it with theory. Various parameters can be singled out for testing.  As the code is written, a loop looks at the effect changing the confocal area(volume) has on the autocorrelation function.  Theory is partly taken from http://fcsxpert.com/classroom/theory/autocorrelation-diffusion-3d.html

#### Functions and Parameters

In [1]:
import numpy as np
import scipy as sp
from numpy.random import normal
from numpy.random import uniform
from numpy.random import poisson
import math
from math import pi
import matplotlib.pyplot as plt
from numpy.linalg import norm
import plotly.graph_objects as go
#
Nt = 100000    # number of time steps
F = .1*Nt      # percentage of the run to collect data
dt = .1        # time step
k_IPlane = 1.0 # envelope constant for Intesity 
k_DPlane=1.0   # envelope constant for dis_Func
C_I = 1.0      # amplitude for Intestity func
C_D=.5         # amplitude for dis_Func
w = .1         # beam wast parameter
# any parameter which needs to be tested can be appended to the function calls and passed through the code where necessary
#
#Function: bleachCalc
#Description: contains a run of a diffusive process for the intensity with bleaching.
#Parmeters: r1 - confocal volume, r2 - particle creation boundary, r3 - absolute boundary, N - particle number, D-diffusion constat.
#Returns: The number of walkers from the bleaching case
#
def bleach(r1,r2,r3,N,D):
    #
    dr = 0.1
    delta = .1
    numDelta = int((r2-r1)/delta)
    p = .1
    N0 = uniform(-r3,r3,(N,2))
    popIt = dt*p*math.pi*(r3**3-r2**3)/6
    P = poisson(popIt,Nt)
    U = np.zeros(numDelta)

    U = np.ones((int(F),int(r3/dr)))
    I = np.zeros((int(F),int(r3/dr)))
    numWalk = np.zeros(int(F))
    for i in range(Nt):
        N0 = step(N0,D,r3)
        N0 = populate(N0,P[i],r2,r3)
        N0 = remove(N0)
        ri=0
        rf=ri+dr
        if (i>=(Nt-int(F))):
            for j in range(int(r3/dr)):
                C = np.logical_and(ri<((N0.T[0]**2)+(N0.T[1]**2))**(.5),((N0.T[0]**2)+(N0.T[1]**2))**(.5)<rf)
                U[i-Nt+int(F)][j] = len(np.where(C))/((annArea(ri,rf)))
                ri+=dr
                rf+=dr
            I[i-Nt+int(F)] = illFunc(N0)
            numWalk[i-Nt+int(F)] = N0.size
    
    print("average illumination for bleaching case: ", np.average(I))
    print()
    print("linear density function for bleaching case at dr = ", dr)
    print(np.average(U,axis = 0))
    print()
    print("number of walkers present: ", np.average(numWalk))
    print()
    return int(np.average(numWalk))
    #
#
#Function: nonBleachCalc
#Description: contains a run of a diffusive process for the intensity without bleaching.
#Parmeters:N - the number of walkers intended for use in the system, D - diffusion constant, r3 - outer boundary to simulation
#Returns: The intensity array
#
def nonBleach(N,D,r3):
    #
    N0 = uniform(-r3,r3,(N,2))
    I = np.zeros(int(F))

    for i in range(Nt):
        N0 = step(N0,D,r3)
        if(i>=(Nt-int(F))):
            I[i-Nt+int(F)] = illFunc(N0)
    print("average illumination for non-bleaching case: ", np.average(I))
    print()
    return I
    #
#
#Function: corr
#Description: Caluclates the correlation function for the intensity.
#Parmeters:I - Intensity array from nonbleach calc.
#Returns: The autocorrelation function 
#
def corr(I):
    #
    S = np.average(I)**2
    autoCor = np.zeros(int(F/2))
    autoCor[0] = np.average(I*I)
    for i in range(1,int(F/2)):
        autoCor[i] = np.average(I[i:]*I[:-i])
    return autoCor
    #
#
#Function: plot_Dev
#Description: Graphs the difference between theoretical result and monte carlo.
#Parmeters:I - Intesnsity array from nonbleach calc, autoCor - autoCorrelation function associated with nonbleaching case
#Returns: dev array representing the graphed data
#
def plt_Dev(I,autoCor,tau,runLbl):
    #
    S = np.average(I)**2
    dev=((1 + (autoCor[0] / S - 1) / ( 1 + np.arange(int(autoCor.size*.02)) / tau) - autoCor[:int(autoCor.size*.02)] / S) / (1 + (autoCor[0] / S - 1) / (1 + np.arange(int(autoCor.size*.02)) / tau)))
    plt.plot(np.arange(int(autoCor.size*.02)),dev, label = runLbl)
    plt.legend()
    return dev
    #
#
#Function: illFunc
#Description: Sums the illumination values associated with the particles to get total intensity.
#Parmeters: A - The array containing particles.
#Returns: Real value corresponding to intensity.
#
# Note: A must be (N,2) and cartesian
#
def illFunc(A):
    #
    return sum(C_I*np.exp(-k_IPlane*(A.T[0]**2))*np.exp(-k_IPlane*(A.T[1]**2)))
    #
#
#Function: disFunc
#Description: gives the value for the disappearance likelihood at each particle position.
#Parmeters: A - The array containing particles.
#Returns: Entire array corresponding to likelihood of dissappearancef-(N,2) dimension.
#
# Note: A must be (N,2) and cartesian
#
def disFunc(A):
    #
    return C_D*np.exp(-k_DPlane*(A.T[0]**2))*np.exp(-k_DPlane*(A.T[1]**2))
    #
#
#Function: step
#Description: adds a varaible step size to each particle.
#Parmeters: A - The list to be stepped, D - diffusion constant, r3 - outer boundary to simulation
#Returns: The new stepped list of particles.
#
# Note: A must be (N,2) and cartesian
#
def step(A,D,r3): 
    #
    X = np.add(A,normal(0,math.sqrt(D*2*dt),(len(A),2)))
    return bound(X,A,r3)
    #
#
#Function: populate
#Description: adds particles according to the population rate.
#Parmeters: A - The list to be stepped, partCnt - the number of particles to add determined by poisson dist, r2 - creation boundary, r3 - outer boundary to simulation
#Returns: The new augmented list of particles.
#
# Note: A must be (2,N) and cartesian
#
def populate(A,partCnt,r2,r3):
    #
    i = 0
    while (i<partCnt):
        a = uniform(-r3,r3,2)
        if r2<norm(a)<r3:
            A = np.vstack((A,a))
            i+=1
    return A 
    #
#
#Function: remove
#Description: Removes particles according to the position being inside a certain radius.
#Parmeters: A - The list to be stepped.
#Returns: The new reduced list of particles.
#
# Note: A must be (2,N) and cartesian
# 
def remove(A):
    #
    bool_M = disFunc(A)*dt<np.random.random(size=A.shape[0])
    C = np.vstack((bool_M,bool_M))                                     
    return np.reshape(A[C.T],(int(A[C.T].size/2),2))
    #
#
#Function: bound
#Description: Prevents particles from moving outside the bounds of the system.
#Parmeters: A - The list to be stepped, B - list holding previous positions, r3 - outer boundary to simulation.
#Returns: The particle list with misbehaving particles at their old positions-in dimension (2,N).
#
# Note: A must be (N,2) and cartesian
#
def bound(A,B,r3):
    #
    c = ((A.T[0]**2)+(A.T[1]**2))**(.5)>r3
    bool_M = np.vstack((c,c)).T
    return np.where(bool_M,B,A)
    #
#
#Function: getArea
#Description: returns the area of an annulus.
#Parmeters:x - lower bound radius of annulus, y - upper bound radius of annulus.
#Returns: double area.
#
def annArea(x,y):
    #
    return pi*y**2-pi*x**2
    #
#

#### Loop and Data Analysis

In [ ]:
#
s = np.zeros((5,2))
D = 1

for i in range(4):
    i+=1
    print("run ", i, ": r1=", .5 * i, " r2=", 2*i, " r3=", 2.5*i, " D=", D, " N=", 5)
    print()
    I = nonBleach(bleach(.5*i,2*i,2.5*i,D,5),D,2.5*i)
    autoCor = corr(I)
    dev = plt_Dev(I,autoCor,w ** 2 / (4 * D),"Run {0}".format(i))
    s[i] = np.average(dev), np.std(dev)


print("(avg,std) table for all 6 runs:")
print(s)
print()
#